In [1]:
"""
Python script to scrape Friktion User Data from Bitquery GraphQL API.

"""

import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists

In [3]:
globalId = "mainnet_income_put_luna"

users = pd.read_csv("luna_put_addresses.csv")

shareprice = pd.read_json(f"../derived_timeseries/{globalId}_sharePricesByGlobalId.json")

shareprice[0]/=1000

In [4]:
res = {}

In [31]:
def calc_amount(addr):
    success = False
    while not success:
        try:
            query = """https://solana-stream-dev-ztbl.ue1-eks-0.prod-czff.zettablock.dev/graphql?query={userTransactions(userAddress:\"%s\"){Hash,Amount,Decimal,InstructionAction,InstructionIndex,InnerIndex,UserAddress,Timestamp,UnixTime,CurrencyName,CurrencyAddress,SenderAddress,SenderTokenMint,ReceiverAddress,UserAction,GlobalID,VaultAuthority,ShareTokenMint,DepositTokenSymbol,DepositTokenCoingeckoId}}"""
            x = pd.DataFrame(json.loads(requests.get(query%addr).content)["userTransactions"])

            success = True
        except Exception as e:
            print("Failed... sleeping for 5")
            time.sleep(5)
            continue
            
    cutoff = 1651802400
    if x.empty:
        return

    df = x.loc[x.UnixTime < cutoff].loc[x.GlobalID == globalId]

    df = df.sort_values("UnixTime")

    shareprice["timestamp"] = shareprice[0].astype('int')
    shareprice["sharePrice"] = shareprice[1]

    asimov = pd.merge_asof(df, shareprice,  left_on="UnixTime", right_on="timestamp", direction='forward')

    pos = asimov.query("UserAction=='Deposit'")
    pos = (pos.Amount/pos.sharePrice.fillna(1)/10**pos.Decimal).sum()

    pos2 = asimov.query("UserAction == 'CancelPendingWithdrawal'")
    if pos2.empty:
        pos2 = 0
    else:
        pos2 = (pos2.Amount/10**pos2.Decimal).sum()

    neg = asimov.query("UserAction=='Withdraw'")
    if neg.empty:
        neg = 0
    else:
        neg = (neg.Amount/10**neg.Decimal).sum()

    neg2 = asimov.query("UserAction=='CancelPendingDeposit'")
    if neg2.empty:
        neg2 = 0
    else:
        neg2= (neg2.Amount/neg2.sharePrice.fillna(1)/10**neg2.Decimal).sum()

    amount = pos+pos2-neg-neg2
    amount = amount if abs(amount) > 0.001 else 0
    print(addr, amount)
    res[addr] = amount

In [32]:
for addr in users.userAddress:
    calc_amount(addr)
    time.sleep(0.5)

4F6J65BByd2eWHJ26RnMnR7vnxPqxNBF9EEJFwSHJsBW 5781.919725879062
DwmePe5umSWvsuQWqcZnbAu75Bm4B1XZCNrMGDxhVEgi 1271.3783778304603
AY5rzzhQkBvNdQx3W25NcmVj58AmpG5CP5NSEUX3GyzH 9.74747963200809
Fm7f6BzAookbmr4m5xaVcQjsF6N3uFMRCkMmWu54k28Q 42.09011592592239
DaKDiSaA7ae4FxHhBbG2AaZ97DMaxU8hjRppNye5mYia 43.28719635169121
9yUokY6Z3iGZ2yMhxPYrZqfnGo2rVAaBEE46ZHKr7Bw7 0
HGwnk76zKpRh2BFvUEjyTThxmFyvX9oTeWo7ouQf1Mqe 98.64039835196641
9bkkeNJL9LjNiWpvNUUPQy3job8KP9KDtNX5DuT7muGP 1943.6179513437419
93KaH8yJpvRz3n1nq7LJuBYbeZzcxWehF7mjY8Kotjw1 965.2620577427483
CZiRHmqePKYP2hnWfuW6EsWsjYrsBdG91tXXXHs84JuF 1719.8071463430315
Ga5K7tiCZPPitmcnR1evPJgPAU8Dm2DNw9unjNq8ABTG 0
65CU53jhtM3abpDXmjrcLt8WZwB7JuuP233xDEgGi5vZ 0
HWstmhmp4DDdZCyU9QgQN2j8PpP5DGwmt23kEsuusuuK 85477.81383166937
3uTMAEZTmR4xFTqjCM7vsRdd8Nw5b5dmz9hrTTYDiRgr 99.58157195421147
C7kUxhQVt1PvaaxCsXi6qF9RP24MmCi9pdxAm4AVRwse 0
ESShcBD34wXp5YtHFKhuC8gb3L4TwXpNcKBgpNFXA9kY 0
HQFH65hxhAwbGbP4pSCV4PJj2Ec2cyrk6c8XmShJaHp2 2.096229419150629
2vQAstU

DYmSowEG5dVNthLg4577BQWnwMhF51EAA5G4Gi3qQPQm 0
4diY1t9kPaqy5nSfJiFeBrDh3YKx9kK84sLgALWVdBUy 0
FrSSqd3AscPDSM37vSzF71Bnt2dDsyCFTR852p7dw3Ys 0
DeB6JtPZhxUcJhCRQHXpn1JEfpDk9PTkKcAqd5tgr1jv 473.8127612574957
J9zW1BogAUG72bMraNbGKzELyjyifUoiFrS2R6YizdDs 7.632364723295861
2kic8gYaNFrvvd1hJvmaHjLQExe4AoY9YUYR8PGFA3xB 7259.619492921812
H8Cvwp6SESYL6J7Ff7XPQzsvW1ewvmfL1XUcdgTDiCLU -486.2196018233594
5yuZ4QunYeTsA1Xn2osyGBLF6zJEGhjhhoRrxv3RLnoi 8.827637957323086
EhBJWJavhWsZ5WuxG2TdmHQ5noUKZHHiqPjawJ7GZTmL 94.54303934703381
8hEeWszgrA2XkRK4GH6zL4Qq5wJBwotwsB6VEweD8YEQ 28784.89612197451
H65e14obSTsuDXcYijLpq1yXjo2ARAUXwQyi2REMsmHy 487.3739816004045
7MFLsQixqAHyMgz414pmg2WLr1tJx7sHFbnX5s5Loikk 0.19494959264016182
9bkkeNJL9LjNiWpvNUUPQy3job8KP9KDtNX5DuT7muGP 1943.6179513437419
FipBUWn6qgFNqxyVMYWebc55cN4TXpfu66tGyivxQQ1Q 965.2620577427483
3xF5EvLaDGUmGNFbYWZ6Bc8u835EanupDdLPuQu6cABW 27258.794668405262
DaKDiSaA7ae4FxHhBbG2AaZ97DMaxU8hjRppNye5mYia 43.28719635169121
C9U2YPcEmsxeJuGhQ233hrRpj6an5rxABg9

4F6J65BByd2eWHJ26RnMnR7vnxPqxNBF9EEJFwSHJsBW 5781.919725879062
CGk38eGmn1qGqXL1BtVS4KHG8LTfyQXypDqnDNvN67hC 956.0475091144868
4F6J65BByd2eWHJ26RnMnR7vnxPqxNBF9EEJFwSHJsBW 5781.919725879062
4jphBN8fmUsocvC4ArMrB9Moicyo1ZuPSNBmGHFpPmbA 9654.059444796098
55HALo7xwMgQsziPinNEZkoELuvGRz6VYFFv78xApf8 969.727216518051
5yuZ4QunYeTsA1Xn2osyGBLF6zJEGhjhhoRrxv3RLnoi 8.827637957323086
HtkkRAayT7nCMRp4LcR8GFTbGZVuJvwSgHdS8j2F3a5X 137.77559064542038
5yuZ4QunYeTsA1Xn2osyGBLF6zJEGhjhhoRrxv3RLnoi 8.827637957323086
74zF7HjPEoG5K5YQSmBToDzQ9vZrbp3orSRoi92UiH9w 503.2341829601001
5zn7ZZ6JfQ8oYxTvsGjMJC8uWk5RW1phQxLThVY3XKMp 0
9qCrsq8Abnq1hfGT2EEW2RbSAmGYnuheEcfkaKhtfenB 10.08572673552335
AxpB8nQ4zRDtoJ6LtnAjKZ4aKJeapin7d1h349fSw1Pe 391.82824961733843
87DMgBLBU5bMBfFbqLN79mn8AhyuctX9jVFxASMw5cG2 96.57192164234716
CUpj211om34hWvYjH7eGvrjA3vbnapGGp22RkQEQkaza 9.931935871877974
73K6QZv2rK6e2eJPa7nBjPRDtajBFVFVujqx8SHk669Y 0
2kic8gYaNFrvvd1hJvmaHjLQExe4AoY9YUYR8PGFA3xB 7259.619492921812
4iLRfA59UtAap3FBzPzL1CJW

HhSTZmUuaFqiQHwHuLUHohg81xktZogygKNpMWrrQAiz 0
7MvdE5D9g8demfTPvo6JTaqbMSvvxo3sfFCrux1qoye4 455.7185294209631
4jfevPD5Q6NYCtBXCxcpxLWw4J2eydt71BB5uPG4eZJb 0
JA96hh8eN2YS6CtZ5GC71kvYA5dZUH6CbkN1osM6tF2M 0
94KFzzXCAvEfSf3jrnUpyN3pne8Xfc6nroB7UxTVqeyB 0
FK92XP14G5pNxqmwYKprc7Q14p11SUaymLEi5592YPnS 236.9129270067469
D2ULM2sYYSjY8UFqZ9vuQw5BiVuoeKUMKQeiiTn5EKXd 0
7H2fjdHYNBottkNDy6x1te27cKBUTtDQfC7E3pgv1f5x 47.375558720100095
AG25ARitey2E2kUoBHSRnUSWLNPErPsvMyGqQCVfQHFG 58047.34124946442
EQcxN92Wgktomfr9DbMkVzXLpqJe9wHrLQKP3wVf7t72 5.000000248751432
Cf554mgKNkLKdn5jCfrGZiuz7oP1CbEmbehm2U5q43nn 0
E1iAeafp9XLQLCu9ErENUxZiNPWGQNCW3F3EDFbpZJvx 2000.0000995005728
6VHKRsV67Hx6zUWdQs8FVNFW7Qb2SBsEd6jqMRD5UVGf 0
7ozEZ85JShXE8nTYsbarXQEzSLcVUxbVBt5SYJtH5niK 1553.9524423095752
mX4KQfX7rA5rsEak8AaqyYipNrPr5q2aw4wMYPwagGF 111.29933250736501
C9Uz6CwHMc9LkGt4SV347k8srr9WMcBT21VKT8KJxeLw 5.000000248751432
253RtmeqfB5NajwSUyn1j8a2vZ4VvUMRE4GnHgRMibpP 0
HfW97Lwm186yH9WWaq1RF322nVyn8vVmhSgy92bjBZkt 710.95970

4YQahZDiDXGtb4TiNVVyFxGt3f1sK2q9KRPBQckr9JQV 0
Xq6EVTy3fV2eLhrcgsvKMhmHiudZKAqFmMErVzNMW7z 686.4323932254032
Xq6EVTy3fV2eLhrcgsvKMhmHiudZKAqFmMErVzNMW7z 686.4323932254032
HsQeQzM2uBgWqZt9oS9wxjqJ8xF8GN6tMFCS6sTW8ocW -484.3247477848055
HRKiSD2AfheGbsSMLUTzU2UNquUE6Uq7QdM7z56Eu8mH 0
BYjKDAtfAh91TgR2r2P1uanwmaquNi2Qjy2jdcNVMffX 0
6gqUqD1dtzAnJsL2tMN9zQgF7QgNMhActEFwM1Nw9LML 17.690701775947673
5KhgmFJkJSnXGZQUVXnorvW1S5aXges8bjgcZ5Nz2G6E 0.8668419897806579
3qUpFZWpEEPvZq5xjnpBKvMHa8sxvjdBwKJ4sACZJ8wt 0
7sGayKe7zWQ72PGRLWfQH1AtMUGkoTiG8swKZCYMmUn8 25282.208145917542
G1CCSvaB12RufyzkkpXNNZ4agYyPR1rBDdT3d6CXGE8k 4708.0337474549715
3dRhTSSGzGFamUkCtFuEbWNXEZQ8RKe92K79dvKczQm6 1985.330328316539
8arRjN8Kg8tZhhkQsoXg5y29DBHKEsxknGuUBqSHUC6i -1944.133076
6BLftRXBEWMypeBv5dYVkQDtUuVg36tAgN42f1EQBBKN 0
Cgfd6Y9rvA2ApgsPPypKsoBw3LyHGNSb19iGTPJgx2Z2 12413.269811151828
J8KH2UnjMYFDE95bQ5ZPDHYd5asieuzetW4yZV8nXmH7 0
99E45zXCJ9vaezroELmD5apbUzczZaccbxVNNfy3SicG 0
GVzLbfVGZrK5MFnmyZgarj47EYu7NZJ76UKQZhD5J3

cyzavZrt2xqCGFQqiJtGYbgUaLmFAPhFyVaitcGkk4p 2.029795907116207
EpoYvnJsRMLuEZ6rt8Mj2t5oUtoykm532jpdVtbEZJgZ 0
E4ps7WZnXL6DgyLpQcFZti5Zf5xDKQKSv7V444m8KZYU 962.0590286114112
CZiRHmqePKYP2hnWfuW6EsWsjYrsBdG91tXXXHs84JuF 1719.8071463430315
4sWCdfFXQqqvJs1TcqoFWaw7Bbou1zGZqQN3s8dSQSsJ -291.7317606221841
mmGdMwUaNpo3dLyc96vaMRLwiPU6djAA2ATynmrHGM6 980.291984506121
7xSX3jvraodTwL2AS8Q5ED3S5mofnN8DaDVih6zb4N8T 321.2599905648741
7xSX3jvraodTwL2AS8Q5ED3S5mofnN8DaDVih6zb4N8T 321.2599905648741
A9rThXw79qaVig1GWyp2qHpzvQEx778B5UvHzvfz3rWf 0
EHWCVPPek2kt512pn6JQDKMdZ7R91gLad24dthVYjUQo 0
EpTJsnmia3h3VKQrZJR5KQrL26qpwXS1VjVBigfk2yb5 3629.1879048034843
FXe9FXTNxvZpzrkQHJwqBHdAydgLCf4ci6dnTz7y3rek 194.53279236059308
5yuZ4QunYeTsA1Xn2osyGBLF6zJEGhjhhoRrxv3RLnoi 8.827637957323086
9B98fmxowLqjNuSqL9kiNP89ZunmwgpJMgeZJCwXBErL 1008.6874071479269
6iDsqe5Yy5UnmbaXt4LVseuAfuPYBuGXHeSqbJ18HaAm 0
C7kUxhQVt1PvaaxCsXi6qF9RP24MmCi9pdxAm4AVRwse 0
H8Cvwp6SESYL6J7Ff7XPQzsvW1ewvmfL1XUcdgTDiCLU -486.2196018233594
36Ffzr

47qfgwuLa8zzHt5K4poSNGPycCkNwHuociwDMFRuUahr 10.000000497502864
4FHJUB3rZ22S9LVwyUsEMfZCGLiKfs719jd1a6tJc3YL 0
DhvMozgnVCx58386fno8QD75JYTfJTnNq2svbW9YwhTc 242.27961970840005
DusP9KWaxZFi61g2G7xctrh9qW2NF4v2S4DD5HLcnDcQ 14601.72448874812
5QrVMjq7hwYwBUGQa1oDtn5Y6mqM6RsWngpVUZAw6a1 0
Eagjgzxfhnmhpd98Hs8vd5ySG18EWrW5WeSk2A2Zphjr 0
AG25ARitey2E2kUoBHSRnUSWLNPErPsvMyGqQCVfQHFG 58047.34124946442
CfGGWEx3cAPLt3yiFptU6B4gDtbSEUJNuEwpCeTTZ3rz 0
C7kUxhQVt1PvaaxCsXi6qF9RP24MmCi9pdxAm4AVRwse 0
6V1mFqD2ZzWzcyGar5Cq19JZjhoSsy1gNaS1CfeMpBX7 499.0252768751432
3qUpFZWpEEPvZq5xjnpBKvMHa8sxvjdBwKJ4sACZJ8wt 0
8XgZw7MLRmCoTgYTic9ZiDCsGW2YoVharQ1cGrMKtWsG 0
Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE 2165534.8007097915
29SinSkCGyFRFWTD3nBTmAMeogFGDT2d5e8p3W1imzKj 0
GVzLbfVGZrK5MFnmyZgarj47EYu7NZJ76UKQZhD5J3wd 945.9494336536864
4e4YMNW54dGfwtCYrpmT9rTQxHjz1xnwM371gnbsZess 284.29551240809633
BQfguZpr4hje4irm9xkcp7yrrKhWgCMCfEB51r3Ecrhg 0
89wbNP1iA1FgxqaZsZkX4htqqvo4PfkvrChHc9p2DfNq 0
7wHuTjBrche1b1dtbjPryoA

In [38]:
len(res)

541

In [39]:
len(list(filter(lambda y: y[1] > 0.01, res.items())))

334

In [40]:
dff = pd.DataFrame(res.items(), columns=["userAddress", "vaultTokens"])

In [51]:
dff = pd.DataFrame(res.items(), columns=["userAddress", "vaultTokens"])

dff.vaultTokens.sum()

POOL = 10366

dff["usdc_send"] = dff.vaultTokens/dff.vaultTokens.loc[dff.index!=363].sum()*POOL

write = dff.loc[dff.index!=363].loc[dff.usdc_send > 5].sort_values("usdc_send")[::-1]

write.to_csv("luna_put_distribution.csv")

In [52]:
write

,userAddress,vaultTokens,usdc_send
438,3KNZ9i1dLNNqpBTKEkTgUQs6TNCd3bzuy6HwfoXACaRs,524725.074727,2430.450234
431,GUTxaEAhD2N4jWfWspBXaF8wFnUPxQhUDuEVab8T24tm,403359.570232,1868.302868
499,GrRG2dYEnLPJZBCxBrAZTsdrKWTCGDHXTJ2Wd6B1Lzjv,292424.388960,1354.467242
404,5SdHHn4Cz5Nb7EVK1jE1fMoBwDnAncCG7FZiHKTrhANu,99199.012538,459.475399
12,HWstmhmp4DDdZCyU9QgQN2j8PpP5DGwmt23kEsuusuuK,85477.813832,395.920802
...,...,...,...
290,CqkjZU58iqDgpruqbKzxMsDhBC3PCx9Q9Gn1cXDmgez8,1409.756608,6.529788
1,DwmePe5umSWvsuQWqcZnbAu75Bm4B1XZCNrMGDxhVEgi,1271.378378,5.888840
195,4qyZRHyT1UrVVpUYJFAK6yYrTBzdWhZThsV3AHTPVRr2,1270.788726,5.886109
211,84DWEA4TiA2eTWv4RTwzdVULUPiPwFiCMmuyyyu5ZPpS,1236.000061,5.724973


In [2]:
147/63000*10366

24.187333333333335

In [2]:
x = pd.read_csv("luna_put_distribution.csv")

In [5]:
x.loc[x.userAddress=="vPdmG4qr2Gvb3fdwYMxbDszpTujrdRifb8PJ57Ni4Jj"]

,Unnamed: 0,userAddress,vaultTokens,usdc_send


In [11]:
y = x.userAddress.values

z = x.usdc_send.values

np.savetxt('luna_put_addresses.log', y, fmt="%s")

np.savetxt('luna_put_amounts.log', z, fmt="%0.2f")

In [6]:
addr = "vPdmG4qr2Gvb3fdwYMxbDszpTujrdRifb8PJ57Ni4Jj"

In [7]:
query = """https://solana-stream-dev-ztbl.ue1-eks-0.prod-czff.zettablock.dev/graphql?query={userTransactions(userAddress:\"%s\"){Hash,Amount,Decimal,InstructionAction,InstructionIndex,InnerIndex,UserAddress,Timestamp,UnixTime,CurrencyName,CurrencyAddress,SenderAddress,SenderTokenMint,ReceiverAddress,UserAction,GlobalID,VaultAuthority,ShareTokenMint,DepositTokenSymbol,DepositTokenCoingeckoId}}"""
x = pd.DataFrame(json.loads(requests.get(query%addr).content)["userTransactions"])

In [8]:
x

,Amount,CurrencyAddress,CurrencyName,Decimal,DepositTokenCoingeckoId,DepositTokenSymbol,GlobalID,Hash,InnerIndex,InstructionAction,InstructionIndex,ReceiverAddress,SenderAddress,SenderTokenMint,ShareTokenMint,Timestamp,UnixTime,UserAction,UserAddress,VaultAuthority
0,7846003664,74ozhToAS71nDVBtFZMMucdkNB95tV2o5fhtFGQeixwS,,6,terrausd,UST,mainnet_income_put_luna,aZ55uHNWQjJaRPZN6RL1o9Fy989oQdcU7Pe2PZ32u75xQb...,1,Burn,1,,vPdmG4qr2Gvb3fdwYMxbDszpTujrdRifb8PJ57Ni4Jj,HS4vScKhtQN48pKcJXZQTUTzctuPRvoK5obfmRR2N1hi,74ozhToAS71nDVBtFZMMucdkNB95tV2o5fhtFGQeixwS,2022-05-08T15:16:57Z,1652023017,Withdraw,vPdmG4qr2Gvb3fdwYMxbDszpTujrdRifb8PJ57Ni4Jj,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
